# Exercice 19 - Pipeline Complet Bronze/Silver/Gold

## Objectifs
- Construire un pipeline ETL complet
- Implementer l'architecture Medallion (Bronze/Silver/Gold)
- Orchestrer les differentes etapes
- Produire des donnees prets pour l'analyse

---

## 1. Architecture du Pipeline

```
+=========================================================================+
|                    PIPELINE MEDALLION COMPLET                           |
+=========================================================================+
|                                                                         |
|  SOURCES                                                                |
|  +-------------+     +-------------+     +-------------+                |
|  | PostgreSQL  |     |    Kafka    |     |   Fichiers  |                |
|  | (Northwind) |     | (Streaming) |     |   (CSV)     |                |
|  +------+------+     +------+------+     +------+------+                |
|         |                   |                   |                       |
|         +-------------------+-------------------+                       |
|                             |                                           |
|                             v                                           |
|  BRONZE (Donnees brutes)                                                |
|  +------------------------------------------------------------------+   |
|  |  s3a://bronze/                                                   |   |
|  |  - customers/    - orders/    - products/    - kafka/            |   |
|  |  Format: Parquet, partition par date d'ingestion                 |   |
|  +------------------------------------------------------------------+   |
|                             |                                           |
|                             v                                           |
|  SILVER (Donnees nettoyees)                                             |
|  +------------------------------------------------------------------+   |
|  |  s3a://silver/                                                   |   |
|  |  - dim_customers/  - dim_products/  - fact_orders/               |   |
|  |  Nettoyage, deduplication, enrichissement                        |   |
|  +------------------------------------------------------------------+   |
|                             |                                           |
|                             v                                           |
|  GOLD (Donnees agregees)                                                |
|  +------------------------------------------------------------------+   |
|  |  s3a://gold/                                                     |   |
|  |  - kpi_ventes/  - analyse_clients/  - rapport_produits/          |   |
|  |  KPIs, aggregations, donnees prets pour dashboards               |   |
|  +------------------------------------------------------------------+   |
|                                                                         |
+=========================================================================+
```

## 2. Configuration

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, current_timestamp, date_format, year, month, dayofmonth,
    when, coalesce, trim, upper, lower, initcap,
    count, sum as spark_sum, avg, max as spark_max, min as spark_min,
    round as spark_round, datediff, current_date,
    row_number, dense_rank, percent_rank
)
from pyspark.sql.window import Window
from datetime import datetime
import os

# --- ETAPE CRITIQUE : Arrêter l'ancienne session pour appliquer le patch ---
try:
    SparkSession.getActiveSession().stop()
    print("🛑 Ancienne session Spark arrêtée avec succès.")
except:
    pass
# --------------------------------------------------------------------------

# Packages nécessaires : PostgreSQL (Source) + S3 (Stockage)
PACKAGES = "org.postgresql:postgresql:42.6.0,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262"

spark = SparkSession.builder \
    .appName("PipelineComplet") \
    .config("spark.jars.packages", PACKAGES) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "10000") \
    .config("spark.hadoop.fs.s3a.socket.timeout", "60000") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Configuration BDD et Dossiers
JDBC_URL = "jdbc:postgresql://postgres:5432/app"
JDBC_PROPS = {"user": "postgres", "password": "postgres", "driver": "org.postgresql.Driver"}

BRONZE = "s3a://bronze"
SILVER = "s3a://silver"
GOLD = "s3a://gold"

DATE_INGESTION = datetime.now().strftime("%Y-%m-%d")

print(f"Session Spark démarrée (Patch '60s' appliqué).")
print(f"Date d'ingestion: {DATE_INGESTION}")

Session Spark démarrée (Patch '60s' appliqué).
Date d'ingestion: 2026-01-18


## 3. BRONZE - Ingestion des donnees brutes

In [2]:
def ingerer_table_bronze(table_name):
    """
    Ingere une table PostgreSQL vers Bronze.
    Ajoute les metadonnees d'ingestion.
    """
    print(f"Ingestion de {table_name}...")
    
    # Lire depuis PostgreSQL
    df = spark.read.jdbc(
        url=JDBC_URL,
        table=table_name,
        properties=JDBC_PROPS
    )
    
    # Ajouter metadonnees
    df_bronze = df \
        .withColumn("_ingestion_timestamp", current_timestamp()) \
        .withColumn("_source", lit("postgresql")) \
        .withColumn("_source_table", lit(table_name))
    
    # Sauvegarder en Bronze
    output_path = f"{BRONZE}/{table_name}/date={DATE_INGESTION}"
    df_bronze.write.mode("overwrite").parquet(output_path)
    
    nb_lignes = df_bronze.count()
    print(f"  -> {nb_lignes} lignes sauvegardees")
    
    return nb_lignes

In [3]:
# Ingerer les tables principales
tables = ["customers", "orders", "order_details", "products", "categories", "employees"]

print("=" * 50)
print("ETAPE BRONZE - Ingestion des donnees brutes")
print("=" * 50)

stats_bronze = {}
for table in tables:
    stats_bronze[table] = ingerer_table_bronze(table)

print("\nResume Bronze:")
for table, count in stats_bronze.items():
    print(f"  {table}: {count} lignes")

ETAPE BRONZE - Ingestion des donnees brutes
Ingestion de customers...


NumberFormatException: For input string: "60s"

## 4. SILVER - Nettoyage et transformation

In [ ]:
# Lire les donnees Bronze
df_customers_raw = spark.read.parquet(f"{BRONZE}/customers/date={DATE_INGESTION}")
df_orders_raw = spark.read.parquet(f"{BRONZE}/orders/date={DATE_INGESTION}")
df_order_details_raw = spark.read.parquet(f"{BRONZE}/order_details/date={DATE_INGESTION}")
df_products_raw = spark.read.parquet(f"{BRONZE}/products/date={DATE_INGESTION}")
df_categories_raw = spark.read.parquet(f"{BRONZE}/categories/date={DATE_INGESTION}")
df_employees_raw = spark.read.parquet(f"{BRONZE}/employees/date={DATE_INGESTION}")

print("Donnees Bronze chargees")

In [ ]:
print("=" * 50)
print("ETAPE SILVER - Nettoyage et transformation")
print("=" * 50)

# --- DIM_CUSTOMERS ---
print("\nCreation de dim_customers...")

df_dim_customers = df_customers_raw \
    .select(
        col("customer_id"),
        initcap(col("company_name")).alias("company_name"),
        initcap(col("contact_name")).alias("contact_name"),
        col("contact_title"),
        trim(col("address")).alias("address"),
        initcap(col("city")).alias("city"),
        upper(col("region")).alias("region"),
        col("postal_code"),
        initcap(col("country")).alias("country"),
        col("phone"),
        col("fax")
    ) \
    .withColumn("_processed_timestamp", current_timestamp()) \
    .dropDuplicates(["customer_id"])

df_dim_customers.write.mode("overwrite").parquet(f"{SILVER}/dim_customers")
print(f"  -> {df_dim_customers.count()} clients")

In [ ]:
# --- DIM_PRODUCTS ---
print("\nCreation de dim_products...")

df_dim_products = df_products_raw \
    .join(df_categories_raw, "category_id", "left") \
    .select(
        col("product_id"),
        initcap(df_products_raw["product_name"]).alias("product_name"),
        col("category_id"),
        initcap(col("category_name")).alias("category_name"),
        col("quantity_per_unit"),
        col("unit_price"),
        col("units_in_stock"),
        col("units_on_order"),
        col("reorder_level"),
        col("discontinued"),
        # Indicateurs derives
        when(col("units_in_stock") < col("reorder_level"), lit("Critique"))
            .when(col("units_in_stock") < col("reorder_level") * 2, lit("Bas"))
            .otherwise(lit("Normal")).alias("stock_status"),
        when(col("unit_price") >= 50, lit("Premium"))
            .when(col("unit_price") >= 20, lit("Standard"))
            .otherwise(lit("Budget")).alias("price_segment")
    ) \
    .withColumn("_processed_timestamp", current_timestamp()) \
    .dropDuplicates(["product_id"])

df_dim_products.write.mode("overwrite").parquet(f"{SILVER}/dim_products")
print(f"  -> {df_dim_products.count()} produits")

In [ ]:
# --- DIM_EMPLOYEES ---
print("\nCreation de dim_employees...")

df_dim_employees = df_employees_raw \
    .select(
        col("employee_id"),
        initcap(col("first_name")).alias("first_name"),
        initcap(col("last_name")).alias("last_name"),
        col("title"),
        col("birth_date"),
        col("hire_date"),
        initcap(col("city")).alias("city"),
        initcap(col("country")).alias("country"),
        col("reports_to")
    ) \
    .withColumn("full_name", 
        concat_ws(" ", col("first_name"), col("last_name"))
    ) \
    .withColumn("_processed_timestamp", current_timestamp())

from pyspark.sql.functions import concat_ws

df_dim_employees = df_employees_raw \
    .select(
        col("employee_id"),
        initcap(col("first_name")).alias("first_name"),
        initcap(col("last_name")).alias("last_name"),
        col("title"),
        col("hire_date"),
        initcap(col("city")).alias("city"),
        initcap(col("country")).alias("country")
    ) \
    .withColumn("full_name", concat_ws(" ", col("first_name"), col("last_name"))) \
    .withColumn("_processed_timestamp", current_timestamp())

df_dim_employees.write.mode("overwrite").parquet(f"{SILVER}/dim_employees")
print(f"  -> {df_dim_employees.count()} employes")

In [ ]:
# --- FACT_ORDERS ---
print("\nCreation de fact_orders...")

df_fact_orders = df_orders_raw \
    .join(df_order_details_raw, "order_id", "inner") \
    .select(
        col("order_id"),
        col("customer_id"),
        col("employee_id"),
        col("order_date"),
        col("required_date"),
        col("shipped_date"),
        col("product_id"),
        col("unit_price"),
        col("quantity"),
        col("discount"),
        # Montants calcules
        (col("unit_price") * col("quantity")).alias("montant_brut"),
        (col("unit_price") * col("quantity") * (1 - col("discount"))).alias("montant_net"),
        (col("unit_price") * col("quantity") * col("discount")).alias("remise"),
        # Dimensions temporelles
        year(col("order_date")).alias("annee"),
        month(col("order_date")).alias("mois"),
        dayofmonth(col("order_date")).alias("jour")
    ) \
    .withColumn("_processed_timestamp", current_timestamp())

df_fact_orders.write.mode("overwrite").parquet(f"{SILVER}/fact_orders")
print(f"  -> {df_fact_orders.count()} lignes de commande")

## 5. GOLD - Agregations et KPIs

In [ ]:
# Charger les donnees Silver
df_customers = spark.read.parquet(f"{SILVER}/dim_customers")
df_products = spark.read.parquet(f"{SILVER}/dim_products")
df_employees = spark.read.parquet(f"{SILVER}/dim_employees")
df_orders = spark.read.parquet(f"{SILVER}/fact_orders")

print("Donnees Silver chargees")
print("=" * 50)
print("ETAPE GOLD - Agregations et KPIs")
print("=" * 50)

In [ ]:
# --- KPI VENTES MENSUELLES ---
print("\nCreation de kpi_ventes_mensuelles...")

df_kpi_ventes = df_orders \
    .groupBy("annee", "mois") \
    .agg(
        count("order_id").alias("nb_commandes"),
        countDistinct("customer_id").alias("nb_clients"),
        spark_sum("montant_net").alias("ca_total"),
        avg("montant_net").alias("panier_moyen"),
        spark_sum("quantity").alias("quantite_totale"),
        spark_sum("remise").alias("remises_totales")
    ) \
    .withColumn("ca_total", spark_round(col("ca_total"), 2)) \
    .withColumn("panier_moyen", spark_round(col("panier_moyen"), 2)) \
    .withColumn("remises_totales", spark_round(col("remises_totales"), 2)) \
    .orderBy("annee", "mois") \
    .withColumn("_generated_timestamp", current_timestamp())

from pyspark.sql.functions import countDistinct

df_kpi_ventes = df_orders \
    .groupBy("annee", "mois") \
    .agg(
        count("order_id").alias("nb_commandes"),
        countDistinct("customer_id").alias("nb_clients"),
        spark_sum("montant_net").alias("ca_total"),
        avg("montant_net").alias("panier_moyen"),
        spark_sum("quantity").alias("quantite_totale"),
        spark_sum("remise").alias("remises_totales")
    ) \
    .withColumn("ca_total", spark_round(col("ca_total"), 2)) \
    .withColumn("panier_moyen", spark_round(col("panier_moyen"), 2)) \
    .withColumn("remises_totales", spark_round(col("remises_totales"), 2)) \
    .orderBy("annee", "mois") \
    .withColumn("_generated_timestamp", current_timestamp())

df_kpi_ventes.write.mode("overwrite").parquet(f"{GOLD}/kpi_ventes_mensuelles")
print(f"  -> {df_kpi_ventes.count()} mois")
df_kpi_ventes.show()

In [ ]:
# --- ANALYSE CLIENTS ---
print("\nCreation de analyse_clients...")

df_analyse_clients = df_orders \
    .groupBy("customer_id") \
    .agg(
        count("order_id").alias("nb_commandes"),
        spark_sum("montant_net").alias("ca_total"),
        avg("montant_net").alias("panier_moyen"),
        spark_min("order_date").alias("premiere_commande"),
        spark_max("order_date").alias("derniere_commande")
    ) \
    .join(df_customers.select("customer_id", "company_name", "country", "city"), "customer_id") \
    .withColumn("ca_total", spark_round(col("ca_total"), 2)) \
    .withColumn("panier_moyen", spark_round(col("panier_moyen"), 2))

# Segmentation RFM simplifiee
window_ca = Window.orderBy(col("ca_total").desc())

df_analyse_clients = df_analyse_clients \
    .withColumn("rang_ca", dense_rank().over(window_ca)) \
    .withColumn("segment",
        when(col("rang_ca") <= 10, lit("VIP"))
        .when(col("rang_ca") <= 30, lit("Premium"))
        .when(col("rang_ca") <= 60, lit("Standard"))
        .otherwise(lit("Occasionnel"))
    ) \
    .withColumn("_generated_timestamp", current_timestamp())

df_analyse_clients.write.mode("overwrite").parquet(f"{GOLD}/analyse_clients")
print(f"  -> {df_analyse_clients.count()} clients analyses")
df_analyse_clients.select(
    "company_name", "country", "nb_commandes", "ca_total", "segment"
).orderBy(col("ca_total").desc()).show(10)

In [ ]:
# --- RAPPORT PRODUITS ---
print("\nCreation de rapport_produits...")

df_rapport_produits = df_orders \
    .groupBy("product_id") \
    .agg(
        count("order_id").alias("nb_ventes"),
        spark_sum("quantity").alias("quantite_vendue"),
        spark_sum("montant_net").alias("ca_produit"),
        avg("unit_price").alias("prix_moyen")
    ) \
    .join(df_products.select(
        "product_id", "product_name", "category_name", 
        "stock_status", "price_segment", "units_in_stock"
    ), "product_id") \
    .withColumn("ca_produit", spark_round(col("ca_produit"), 2)) \
    .withColumn("prix_moyen", spark_round(col("prix_moyen"), 2))

# Ranking par categorie
window_cat = Window.partitionBy("category_name").orderBy(col("ca_produit").desc())

df_rapport_produits = df_rapport_produits \
    .withColumn("rang_categorie", row_number().over(window_cat)) \
    .withColumn("_generated_timestamp", current_timestamp())

df_rapport_produits.write.mode("overwrite").parquet(f"{GOLD}/rapport_produits")
print(f"  -> {df_rapport_produits.count()} produits analyses")
df_rapport_produits.select(
    "product_name", "category_name", "quantite_vendue", "ca_produit", "rang_categorie"
).orderBy(col("ca_produit").desc()).show(10)

In [ ]:
# --- PERFORMANCE EMPLOYES ---
print("\nCreation de performance_employes...")

df_perf_employes = df_orders \
    .groupBy("employee_id") \
    .agg(
        countDistinct("order_id").alias("nb_commandes"),
        countDistinct("customer_id").alias("nb_clients"),
        spark_sum("montant_net").alias("ca_genere"),
        avg("montant_net").alias("ca_moyen_ligne")
    ) \
    .join(df_employees.select("employee_id", "full_name", "title", "city"), "employee_id") \
    .withColumn("ca_genere", spark_round(col("ca_genere"), 2)) \
    .withColumn("ca_moyen_ligne", spark_round(col("ca_moyen_ligne"), 2)) \
    .withColumn("_generated_timestamp", current_timestamp())

df_perf_employes.write.mode("overwrite").parquet(f"{GOLD}/performance_employes")
print(f"  -> {df_perf_employes.count()} employes")
df_perf_employes.orderBy(col("ca_genere").desc()).show()

## 6. Verification du pipeline

In [ ]:
print("=" * 60)
print("VERIFICATION DU PIPELINE")
print("=" * 60)

# Verifier Bronze
print("\nBRONZE:")
for table in tables:
    try:
        count = spark.read.parquet(f"{BRONZE}/{table}/date={DATE_INGESTION}").count()
        print(f"  {table}: {count} lignes")
    except:
        print(f"  {table}: ERREUR")

# Verifier Silver
print("\nSILVER:")
for dim in ["dim_customers", "dim_products", "dim_employees", "fact_orders"]:
    try:
        count = spark.read.parquet(f"{SILVER}/{dim}").count()
        print(f"  {dim}: {count} lignes")
    except:
        print(f"  {dim}: ERREUR")

# Verifier Gold
print("\nGOLD:")
for kpi in ["kpi_ventes_mensuelles", "analyse_clients", "rapport_produits", "performance_employes"]:
    try:
        count = spark.read.parquet(f"{GOLD}/{kpi}").count()
        print(f"  {kpi}: {count} lignes")
    except:
        print(f"  {kpi}: ERREUR")

In [ ]:
# Resume executif
print("\n" + "=" * 60)
print("RESUME EXECUTIF")
print("=" * 60)

# KPIs globaux
df_kpi = spark.read.parquet(f"{GOLD}/kpi_ventes_mensuelles")
total_ca = df_kpi.agg(spark_sum("ca_total")).collect()[0][0]
total_commandes = df_kpi.agg(spark_sum("nb_commandes")).collect()[0][0]

df_clients = spark.read.parquet(f"{GOLD}/analyse_clients")
nb_clients = df_clients.count()
nb_vip = df_clients.filter(col("segment") == "VIP").count()

print(f"\nChiffre d'affaires total: {total_ca:,.2f} EUR")
print(f"Nombre de commandes: {total_commandes:,}")
print(f"Nombre de clients: {nb_clients}")
print(f"Clients VIP: {nb_vip}")
print(f"\nPipeline execute le: {DATE_INGESTION}")

---

## Exercice

**Objectif** : Etendre le pipeline

**Consigne** :
1. Ajoutez une table Gold "analyse_geographique" avec le CA par pays
2. Calculez la croissance mensuelle du CA
3. Identifiez le top 3 des pays par CA

A vous de jouer :

In [ ]:
# TODO: Creer analyse_geographique
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as spark_sum, col, desc, lag, round as spark_round, current_timestamp, count

# --- 1. Création de la table Gold : Analyse Géographique ---
print("Construction de l'analyse géographique...")

# On a besoin de joindre les Ventes (Fact) avec les Clients (Dim) pour récupérer le pays
df_geo_analysis = df_orders.join(df_customers, "customer_id") \
    .groupBy("country") \
    .agg(
        spark_sum("montant_net").alias("ca_total"),
        count("order_id").alias("nb_commandes"),
        avg("montant_net").alias("panier_moyen")
    ) \
    .withColumn("ca_total", spark_round(col("ca_total"), 2)) \
    .withColumn("panier_moyen", spark_round(col("panier_moyen"), 2)) \
    .withColumn("_generated_at", current_timestamp())

# Sauvegarde dans le Data Lake (Gold)
df_geo_analysis.write.mode("overwrite").parquet(f"{GOLD}/analyse_geographique")
print(f"Table 'analyse_geographique' sauvegardée.")


# --- 2. Top 3 des Pays par CA ---
print("\n--- TOP 3 PAYS PAR CHIFFRE D'AFFAIRES ---")
df_geo_analysis.select("country", "nb_commandes", "ca_total") \
    .orderBy(desc("ca_total")) \
    .show(3)


# --- 3. Calcul de la Croissance Mensuelle (Window Function) ---
print("\n--- CROISSANCE MENSUELLE DU CA ---")

# Pour cela, on réutilise la table 'kpi_ventes_mensuelles' qu'on a créée plus haut
df_mensuel = spark.read.parquet(f"{GOLD}/kpi_ventes_mensuelles")

# On définit une fenêtre ordonnée par temps pour comparer avec le mois précédent
window_spec = Window.orderBy("annee", "mois")

df_croissance = df_mensuel.withColumn(
    "ca_precedent", 
    lag("ca_total", 1).over(window_spec)
).withColumn(
    "croissance_pct", 
    spark_round(((col("ca_total") - col("ca_precedent")) / col("ca_precedent")) * 100, 2)
).select("annee", "mois", "ca_total", "ca_precedent", "croissance_pct")

df_croissance.orderBy("annee", "mois").show()

---

## Resume

Dans ce notebook, vous avez appris :
- Comment construire un **pipeline ETL complet**
- L'architecture **Medallion** (Bronze/Silver/Gold)
- L'**ingestion** depuis PostgreSQL vers Bronze
- Le **nettoyage et transformation** vers Silver
- La creation de **KPIs et agregations** dans Gold
- La **verification** du pipeline

Ce pipeline est la base de toute architecture Data Lake moderne.